In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#read csv that contains info for people in the industry
talent = pd.read_csv('zippedData/imdb.name.basics.csv.gz')

In [3]:
talent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


In [4]:
#investigate profession
talent['primary_profession'].sample(10)

388456    camera_department,sound_department,cinematogra...
51513        editor,editorial_department,production_manager
438936                                             composer
409402                                              actress
241996                                                  NaN
190696                               producer,miscellaneous
333412                                             composer
50606                                          writer,actor
428050                          art_department,art_director
213030                                      producer,writer
Name: primary_profession, dtype: object

In [5]:
#import data from movies WITH $
movies = pd.read_csv('zippedData/movie_data.csv',index_col=[0])
movies['d_roi'] = movies['domestic_gross'] / movies['production_budget']

In [6]:
movies.shape

(1371, 10)

In [7]:
genre = movies
split = genre['genres'].str.split(",", expand = True)
genre['one']= split[0]
genre['two']= split[1]
genre['three']= split[2]
g1 = genre[['tconst','averagerating', 'numvotes', 'production_budget', 'domestic_gross', 'worldwide_gross',"d_roi", 'one']].rename(columns={'one':'genre'})
g2 = genre[['tconst','averagerating', 'numvotes', 'production_budget', 'domestic_gross', 'worldwide_gross',"d_roi", 'two']].rename(columns={'two':'genre'})
g3 = genre[['tconst','averagerating', 'numvotes', 'production_budget', 'domestic_gross', 'worldwide_gross',"d_roi", 'three']].rename(columns={'three':'genre'})
genres_list = [g1, g2, g3]
genres = pd.concat(genres_list)
genres.dropna(inplace=True)
genres.head()

,tconst,averagerating,numvotes,production_budget,domestic_gross,worldwide_gross,d_roi,genre
0,tt1542344,7.6,323949,18000000.0,18335230.0,6.021717e+07,1.018624,Adventure
1,tt1484522,7.0,1190,2500000.0,100280.0,1.002800e+05,0.040112,Documentary
2,tt1179056,5.2,85497,35000000.0,63075011.0,1.177296e+08,1.802143,Crime
3,tt1014759,6.5,358813,200000000.0,334191110.0,1.025491e+09,1.670956,Adventure
4,tt1175709,6.3,49684,20000000.0,582024.0,8.736170e+05,0.029101,Crime


In [8]:
#find top documentaries by ratings
doc = pd.read_csv('zippedData/imdb_merge.csv',index_col=[0])
doc

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,merge_id
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,2013 Sunghursh
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,2019 One Day Before the Rainy Season
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,2018 The Other Side of the Wind
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,2018 Sabse Bada Sukh
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,2017 The Wandering Soap Opera
...,...,...,...,...,...,...,...,...,...
73849,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130.0,Drama,8.4,365,2019 Padmavyuhathile Abhimanyu
73850,tt9913056,Swarm Season,Swarm Season,2019,86.0,Documentary,6.2,5,2019 Swarm Season
73851,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,6.2,6,2019 Diabolik sono io
73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136,2019 Sokagin Çocuklari


In [9]:
#separating genres
genre = doc
split = genre['genres'].str.split(",", expand = True)
genre['one']= split[0]
genre['two']= split[1]
genre['three']= split[2]
g1 = genre[['tconst','averagerating', 'numvotes', 'start_year','one']].rename(columns={'one':'genre'})
g2 = genre[['tconst','averagerating', 'numvotes', 'start_year','two']].rename(columns={'two':'genre'})
g3 = genre[['tconst','averagerating', 'numvotes', 'start_year','three']].rename(columns={'three':'genre'})
genres_list = [g1, g2, g3]
doc = pd.concat(genres_list)
doc.dropna(inplace=True)
doc

,tconst,averagerating,numvotes,start_year,genre
0,tt0063540,7.0,77,2013,Action
1,tt0066787,7.2,43,2019,Biography
2,tt0069049,6.9,4517,2018,Drama
3,tt0069204,6.1,13,2018,Comedy
4,tt0100275,6.5,119,2017,Comedy
...,...,...,...,...,...
73819,tt9889926,5.2,13,2018,Romance
73820,tt9889952,8.0,12,2018,Romance
73821,tt9890030,7.4,13,2017,Romance
73827,tt9896916,7.9,61,2019,Family


In [10]:
#dataframe of documentaries with their ratings
doc = doc[doc['genre']=="Documentary"]

In [11]:
#use talent dataframe to find people who worked on movies 
#in the doc dataframe
talent.shape

(606648, 6)

In [12]:
#split known titles to their own rows so there is only one tconst on a row
doc_talent = talent
split = talent['known_for_titles'].str.split(",", expand = True)
talent['one']= split[0]
talent['two']= split[1]
talent['three']= split[2]
talent['four']= split[3]
g1 = talent[['nconst','primary_name', 'primary_profession', 'one']].rename(columns={'one':'tconst'})
g2 = talent[['nconst','primary_name', 'primary_profession', 'two']].rename(columns={'two':'tconst'})
g3 = talent[['nconst','primary_name', 'primary_profession', 'three']].rename(columns={'three':'tconst'})
g4 = talent[['nconst','primary_name', 'primary_profession', 'four']].rename(columns={'four':'tconst'})
genres_list = [g1, g2, g3, g4]
doc_talent = pd.concat(genres_list)
doc_talent.dropna(inplace=True)
doc_talent

,nconst,primary_name,primary_profession,tconst
0,nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer",tt0837562
1,nm0061865,Joseph Bauer,"composer,music_department,sound_department",tt0896534
2,nm0062070,Bruce Baum,"miscellaneous,actor,writer",tt1470654
3,nm0062195,Axel Baumann,"camera_department,cinematographer,art_department",tt0114371
4,nm0062798,Pete Baxter,"production_designer,art_department,set_decorator",tt0452644
...,...,...,...,...
606579,nm9942718,Witri Asmaida,actress,tt8638476
606581,nm9943777,Jeremy Bradshaw,"actor,cinematographer,producer",tt9133458
606593,nm9951829,Hsin-Yen Hsu,actress,tt6270494
606628,nm9980896,Rama Narayanan,"director,writer",tt8929818


In [13]:
#join doc and doc talent 
docs_joined = doc_talent.set_index('tconst').join(doc.set_index('tconst'))

In [14]:
docs_joined.dropna(subset=['averagerating'],inplace=True)

In [15]:
docs_joined

,nconst,primary_name,primary_profession,averagerating,numvotes,start_year,genre
tconst,,,,,,,
tt0159369,nm1347778,William Welles,"editor,actor,visual_effects",7.6,53.0,2013.0,Documentary
tt0159369,nm2042141,Shannon Mulholland,producer,7.6,53.0,2013.0,Documentary
tt0159369,nm0611850,John Mulholland,"director,writer,producer",7.6,53.0,2013.0,Documentary
tt0159369,nm0186908,Nancy Crawford,actress,7.6,53.0,2013.0,Documentary
tt0159369,nm2495087,Richard Zampella,"producer,editor",7.6,53.0,2013.0,Documentary
...,...,...,...,...,...,...,...
tt9904844,nm4256971,Israel Bañuelos,"sound_department,composer,cinematographer",8.7,218.0,2019.0,Documentary
tt9904844,nm7375115,Garry Ferrier,"composer,music_department",8.7,218.0,2019.0,Documentary
tt9905476,nm10523880,Steve Gherebean,"editor,producer,director",9.3,13.0,2019.0,Documentary


In [16]:
docs_joined.describe()

,averagerating,numvotes,start_year
count,56994.000000,56994.000000,56994.000000
mean,7.408783,527.672071,2014.096028
std,1.039426,3126.980653,2.487205
min,1.000000,5.000000,2010.000000
25%,6.800000,10.000000,2012.000000
50%,7.400000,25.000000,2014.000000
75%,8.100000,110.000000,2016.000000
max,10.000000,74978.000000,2019.000000


In [17]:
#sort the list for the top documentaries (>75%)
top_docs = docs_joined[docs_joined['averagerating']>8.1]

In [18]:
#split professions to find directors and producers 
split = top_docs['primary_profession'].str.split(",", expand = True)
top_docs['one']= split[0]
top_docs['two']= split[1]
top_docs['three']= split[2]
g1 = top_docs[['nconst','primary_name', 'averagerating', 'start_year','one']].rename(columns={'one':'profession'})
g2 = top_docs[['nconst','primary_name', 'averagerating', 'start_year','two']].rename(columns={'two':'profession'})
g3 = top_docs[['nconst','primary_name', 'averagerating', 'start_year','three']].rename(columns={'three':'profession'})
jobs_list = [g1, g2, g3]
top_docs = pd.concat(jobs_list)
top_docs.dropna(inplace=True)
top_docs

<ipython-input-18-c8d2f842918e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_docs['one']= split[0]
<ipython-input-18-c8d2f842918e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_docs['two']= split[1]
<ipython-input-18-c8d2f842918e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

,nconst,primary_name,averagerating,start_year,profession
tconst,,,,,
tt0430524,nm1749018,Ermanno Vanino,8.7,2011.0,cinematographer
tt0436782,nm0074206,Howard S. Berger,9.1,2013.0,director
tt0436782,nm5362951,Joseph L. Bagamery,9.1,2013.0,producer
tt0436782,nm5362958,Nora Moss,9.1,2013.0,producer
tt0436782,nm1167800,Susan Stahman,9.1,2013.0,producer
...,...,...,...,...,...
tt9899290,nm3305827,Jason S. Berlin,8.7,2019.0,producer
tt9899290,nm2243354,Adam Avagliano,8.7,2019.0,miscellaneous
tt9904844,nm4256971,Israel Bañuelos,8.7,2019.0,cinematographer


In [27]:
#create a list of directors from recent years with top ratings
top_docs = top_docs[(top_docs['profession']=='director') & \
                    (top_docs['averagerating'] > 9.7) & \
                   (top_docs['start_year'] > 2016)]

In [28]:
top_docs.sort_values(by='averagerating',ascending=False)

,nconst,primary_name,averagerating,start_year,profession
tconst,,,,,
tt7227500,nm1249950,Robert Tutak,10.0,2018.0,director
tt8730716,nm4568586,Taylor Morden,10.0,2019.0,director
tt5089804,nm7633303,Lindsay Thompson,10.0,2019.0,director
tt7227500,nm5335596,Saud Jubaer,10.0,2018.0,director
tt9743544,nm10174241,Andrew Jezard,9.9,2018.0,director
tt9747294,nm3441491,Jesse Gilbert,9.9,2019.0,director
tt4023464,nm6757200,Nick Saporito,9.8,2018.0,director
tt4023464,nm8459774,Jonathan Turcotte,9.8,2018.0,director
tt4818518,nm0359087,Lynn Hamrick,9.8,2018.0,director
